In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load the data
file_path = r'../results/FED_PROTEIN_FINAL_ULTIMATE_DATA_WITHOUT_IMI_IPI_LISTS.csv'
data = pd.read_csv(file_path)

# Function to unpack list-like columns into individual day columns
def unpack_days(df, column_prefix):
    unpacked_cols = []
    for col in df.columns:
        if column_prefix in col:
            # Unpack the list into separate columns
            unpacked = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)
            max_len = max(unpacked.apply(len))
            for i in range(max_len):
                day_col = f"{col}_day_{i+1}"
                df[day_col] = unpacked.apply(lambda x: x[i] if len(x) > i else np.nan)
                unpacked_cols.append(day_col)
    return df, unpacked_cols

# Unpack relevant list columns for each meal/snack/mega meal per day
columns_to_unpack = [
    'all_mega_meals_per_day', 'all_pellets_per_day',
    'all_meals_per_day', 'all_snacks_per_day'
]

# Unpack the day-wise data
for col in columns_to_unpack:
    data, _ = unpack_days(data, col)

# Exclude hourly columns (filter out columns with 'hourly' in their name)
columns_to_exclude = [col for col in data.columns if 'hourly' in col]
data = data.drop(columns=columns_to_exclude)

# Perform ANOVA for multiple components
components = ['grain_meal_size', 'grain_snack_size', 'grain_number_of_meals', 
              'grain_number_of_snacks', 'grain_mega_meal_frequency']

anova_results = {}

for component in components:
    try:
        # Fit the ANOVA model with sex, order, and interaction terms
        # Since we don't have a column named 'diet_phase', we use 'grain', 'pr', 'nr' phases implicitly
        formula = f'{component} ~ C(sex) * C(order)'  # Interaction of sex and order (diet phase can be added later if needed)
        model = ols(formula, data=data).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        anova_results[component] = anova_table
    except Exception as e:
        print(f"Could not process ANOVA for {component}: {e}")

# Descriptive statistics for the dataset
descriptive_stats = data.describe()

# Display the descriptive statistics
print("Descriptive Statistics:")
print(descriptive_stats)

# Output ANOVA tables for each component
for component, anova_table in anova_results.items():
    print(f"\nANOVA results for {component}:")
    print(anova_table)


Descriptive Statistics:
           order  grain_meal_size  grain_snack_size  grain_number_of_meals  \
count  23.000000        23.000000              23.0              23.000000   
mean    1.521739         2.896004               1.0              80.478261   
std     0.510754         0.105025               0.0              28.124511   
min     1.000000         2.716216               1.0              31.000000   
25%     1.000000         2.836067               1.0              61.000000   
50%     2.000000         2.895652               1.0              80.000000   
75%     2.000000         2.956617               1.0             104.500000   
max     2.000000         3.098361               1.0             127.000000   

       grain_meal_frequency  grain_number_of_snacks  grain_snack_frequency  \
count             23.000000               23.000000              23.000000   
mean               1.193044               47.695652               0.704957   
std                0.407931            